In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model

In [ ]:
train_dir = "/content/train" # Replace with your training directory
test_dir = "/content/test"  # Replace with your test directory

class_names = os.listdir(train_dir)
num_classes = len(class_names)

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,   
)

train_data = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=32,
    seed=42,
    subset="training",
)

val_data = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(150, 150),
    class_mode='categorical',
    batch_size=32,
    seed=42,
    subset="validation",
)

In [ ]:
'''
test_datagen = ImageDataGenerator(rescale=1./255,)

test_data = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(150, 150),
    batch_size=32,
    seed=42,
)'''

In [ ]:
train_data.class_indices

In [ ]:
val_data.class_indices

In [ ]:
def create_model(input_shape = (150, 150, 3)):

    Inputs = L.Input(shape=input_shape)
    x = L.Conv2D(512, 3, padding="same")(Inputs)
    x = L.MaxPooling2D((2,2))(x)
    x = L.Activation("relu")(x)
    x = L.Conv2D(256, 3, padding="same")(Inputs)
    x = L.MaxPooling2D((2,2))(x)
    x = L.Activation("relu")(x)
    x = L.Conv2D(128, 3, padding="same")(x)
    x = L.MaxPooling2D((2,2))(x)
    x = L.Activation("relu")(x)
    x = L.Conv2D(64, 3, padding="same")(x)
    x = L.MaxPooling2D((2,2))(x)
    x = L.Activation("relu")(x)
    x = L.Flatten()(x)
    x = L.Dense(256, activation="relu")(x)
    if num_classes > 2:
        Outputs = L.Dense(num_classes, activation="softmax")(x)
        model = Model(Inputs, Outputs, name="Model")
        model.compile(
            loss = "categorical_crossentropy",
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            metrics=["accuracy"]
        )
    else:
        Outputs = L.Dense(num_classes, activation="sigmoid")(x)
        model = Model(Inputs, Outputs, name="Model")
        model.compile(
            loss = "binary_crossentropy",
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            metrics=["accuracy"]
        )

    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)

In [ ]:
history = model.fit(train_data,
    epochs=32,
    validation_data = val_data,
    callbacks = [early_stopping]
)

In [ ]:
# Plot the validation and training data separately
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
plot_loss_curves(history)

In [ ]:
# Initialize lists to store filenames and predicted classes
filenames = []
predictions = []

# Iterate over the images in the folder
for filename in os.listdir(test_dir):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load and preprocess the image
        image_path = os.path.join(test_dir, filename)
        image = Image.open(image_path)
        image = image.resize((150, 150))
        image = np.array(image)
        image = np.expand_dims(image, axis=0)
        image = image / 255.0

        # Make prediction
        prediction = model.predict(image)
        predicted_class = np.argmax(prediction)  # Get the index of the highest probability class

        # Extract filename without extension
        filename_no_ext = os.path.splitext(filename)[0]

        # Append to lists
        filenames.append(filename_no_ext)
        predictions.append(predicted_class)

# Create a DataFrame
data = {'ID': filenames, 'LABEL': predictions}
df = pd.DataFrame(data)

# Write DataFrame to CSV
df.to_csv("/content/Submission-Final.csv", index=False)